#### Music Generation with Python | Part 2

In [46]:
from music21 import note, chord, stream
import random

In [51]:
class MinorMusicGenerator:
    # scale is a number from 59 (B) to 70 (Bb)
    def __init__(self, scale=60):
        if isinstance(scale, int) == False:
            raise("scale must be an integer")
        elif scale < 59 or scale > 70:
                raise("scale must be in a range from 59 to 70")
        else:
             self.scale = scale
        self.correct_notes()
        self.correct_minor_chords()
        self.create_baselines()

    def correct_notes(self) -> list:
        shifts: list[int] = [0, 2, 3, 5, 7, 8, 10]
        notes = [(self.scale + shift) for shift in shifts]
        self.correct_notes = notes
    
    @classmethod
    def get_minor_chord(cls, note, duration: int = 4):
        new_chord = chord.Chord([note, note + 3, note + 7], quarterLength=duration)
        return new_chord
    
    def correct_minor_chords(self) -> list:
        first_chord = self.get_minor_chord(self.scale - 12)
        second_chord = self.get_minor_chord(self.scale + 5 - 12)
        third_chord = self.get_minor_chord(self.scale + 7 - 12)
        self.minor_chords = [first_chord, second_chord, third_chord]

    def create_one_baseline(self, scale): # 12 notes separetely: Scale - 24 ... Scale + 24 ... Scale - 21
        cur_note = scale - 24
        return [cur_note, cur_note + 3, cur_note + 7, cur_note + 12,
                 cur_note + 15, cur_note + 19, cur_note + 24, cur_note + 19,
                 cur_note + 15, cur_note + 12, cur_note + 7, cur_note + 3]
    
    def create_baselines(self):
         first_baseline = self.create_one_baseline(self.scale)
         second_baseline = self.create_one_baseline(self.scale + 5)
         third_baseline = self.create_one_baseline(self.scale + 7)
         self.baselines = [first_baseline, second_baseline, third_baseline]

In [53]:
new_song_generator = MinorMusicGenerator(60)

myStream = stream.Stream()
note_duration = [2, 1, 0.5, 0.25]
intervals = 20

def add_one_interval(index: int, octave_shift_from_4: int = 0, velocity: int = 90):
    # generating notes for the right hand
    random_number = random.randint(0, 3)
    number_of_notes = 2 ** random_number
    duration = note_duration[random_number]
    shift: int = octave_shift_from_4 * 12
    for i in range(number_of_notes):
        random_note = new_song_generator.correct_notes[random.randint(0, 6)] + shift
        my_note = note.Note(random_note, quarterLength=duration)
        my_note.volume.velocity = velocity
        myStream.append(my_note)

    # generating the chord for the left hand
    random_chord = new_song_generator.minor_chords[random.randint(0, 2)]
    newChord = chord.Chord(random_chord.pitches)
    newChord.volume.velocity = 60
    myStream.insert(index, newChord)

for i in range(intervals):
    add_one_interval(2 * i, octave_shift_from_4=random.randint(-1, 0), velocity=random.randint(70, 120))
add_one_interval(2 * intervals, velocity=50)

In [54]:
myStream.write('midi', fp='try.midi')

'try.midi'

#### Saving notes (sheet music)

In [107]:
from music21 import converter, stream

# Replace 'your_file.mid' with the path to your MIDI file
midi_file_path = 'try.midi'

# Parse the MIDI file
midi_score = converter.parse(midi_file_path)

# Create a stream.Score object
score = stream.Score()

# Add the parsed MIDI score to the new Score object
score.append(midi_score)

# Replace 'output_score.xml' with the desired output sheet music file path and format
output_sheet_music_path = 'output_score.xml'

# Write the Score object to a sheet music file (in MusicXML format)
score.write('xml', fp=output_sheet_music_path)

/home/nick/.local/lib/python3.11/site-packages/music21/musicxml/m21ToXml.py:531: MusicXMLWarning: <music21.stream.Score 0x7f0b546ec110> is not well-formed; see isWellFormedNotation()
  warnings.warn(f'{sc} is not well-formed; see isWellFormedNotation()',


PosixPath('/home/nick/Desktop/YouTube/videos/video 7/output_score.xml')

## Generator with a simple baseline

In [ ]:
# interval: quarterlength = 4
myStream = stream.Stream()
note_duration = [2, 1, 0.5]
intervals = 20
volumes = [100, 50, 60, 60, 70, 80, 100, 80, 70, 60, 50, 50]

def add_one_interval(index: int, octave_shift_from_4: int = 0, velocity: int = 90):
    # generating notes for the right hand
    random_number = random.randint(0, 2)
    number_of_notes = 2 ** (random_number + 1)
    duration = note_duration[random_number]
    shift: int = octave_shift_from_4 * 12
    for i in range(number_of_notes):
        random_note = new_song_generator.correct_notes[random.randint(0, 6)] + shift
        my_note = note.Note(random_note, quarterLength=duration)
        my_note.volume.velocity = velocity
        myStream.append(my_note)

    # generating the chord for the left hand
    sequence_of_notes = new_song_generator.baselines[random.randint(0, 2)]
    for i in range(0, 12):
        cur_note = sequence_of_notes[i]
        new_note = note.Note(cur_note, quarterLength=0.33)
        new_note.volume.velocity = velocity
        new_note.volume.velocity = volumes[i]
        myStream.insert(index + (i / 3), new_note)

for i in range(intervals):
    add_one_interval(4 * i, octave_shift_from_4=random.randint(-1, 0), velocity=random.randint(70, 120))
add_one_interval(2 * intervals, velocity=50)